### Example Dataset and Resource Requirements

This example data is a downsampled version of the real dataset, used solely to demonstrate the model's workflow.

- **Memory Requirements**:
  - A minimum of **64GB** of memory is recommended to ensure smooth data processing and model training.
  - In memory-limited environments, the memory requirement can be reduced by downsizing the dataset (e.g., selecting a smaller cell_size or reducing the number of neighbors to decrease the subgraph size).

In [1]:
import os
import time
import math
import gzip
import re
import numpy as np
import pandas as pd
import EmitGCL 
from EmitGCL.utils import *
from EmitGCL.loss_function import *
from EmitGCL.conv import *
from EmitGCL.emitgcl_model import *

In [2]:
def get_cancer_metastasis_genes():
    kegg = KEGG()

    # Define cancer metastasis pathways
    cancer_metastasis_pathways = {
        'Protein processing in endoplasmic reticulum': 'hsa04141',
        'mTOR signaling pathway': 'hsa04150',
        'NF-kappa B signaling pathway': 'hsa04064',
        'Autophagy': 'hsa04140',
        'p53 signaling pathway': 'hsa04115',
        'Apoptosis': 'hsa04210'
    }


    pathway_genes = {}

    # Iterate over each pathway
    for pathway_name, pathway_id in cancer_metastasis_pathways.items():
        # Retrieve pathway information
        pathway_info = kegg.get(pathway_id)
        parsed_pathway = kegg.parse(pathway_info)

        # Get and store the gene list
        genes = parsed_pathway['GENE']
        gene_symbols = []
        for gene_id, gene_info in genes.items():
            # Get the gene symbol
            gene_symbol = gene_info.split(' ')[0].split(';')[0]
            gene_symbols.append(gene_symbol)

        pathway_genes[pathway_name] = gene_symbols

    return pathway_genes

pathway_genes = get_cancer_metastasis_genes()

### Data Preprocessing

- Construct separate subgraphs for primary and metastatic sites.

In [3]:
import numpy as np
import pandas as pd
from scipy.sparse import vstack, csr_matrix
# Main execution
if __name__ == "__main__":

    ignore_warnings()
    set_random_seed()

    input_directory = '../Data'
    
    # Parses command-line arguments for training a GNN on a gene cell graph, you can modify it as needed.
    (output_file, labsm, lr, wd, n_hid, nheads, nlayers, 
     cell_size, neighbor, egrn) = parse_arguments(egrn=True, output_file= f'./{input_directory}/Result/')
    
    # Get the genes associated with the pathways
    pathway_genes = get_cancer_metastasis_genes()
    
    output_file = f'./{input_directory}/Result/'  # relative path for saving results
    attention_file = f'./{input_directory}/Result/Attention/'
    
    ensure_dir(output_file)
    ensure_dir(attention_file)
    
    # Load the saved AnnData object
    input_file = "../Data/combined_data.h5ad"
    adata_combined = sc.read(input_file)
    
    gene_names = adata_combined.var_names
    cell_names = adata_combined.obs_names

    labels = adata_combined.obs['label']
    
    # Extract the data matrix (genes x cells)
    cell_counts_matrix = adata_combined.X
    
    # Convert the matrix to CSR format
    cell_counts_matrix_csr = csr_matrix(cell_counts_matrix)
    
    # Transpose the matrix so that genes are columns and cells are rows
    gene_cell = cell_counts_matrix_csr.transpose()
    
    # Set observation names (cells) and variable names (genes)
    gene_cell.obs_names = adata_combined.obs_names  # Cells are now the rows (after transpose)
    gene_cell.var_names = adata_combined.var_names  # Genes are now the columns (after transpose)

    RNA_matrix = gene_cell

    cell_num = RNA_matrix.shape[1]
    gene_num = RNA_matrix.shape[0]

    print(f"Cell number: {cell_num}")
    print(f"Gene number: {gene_num}")

    device = torch.device("cuda" if cuda.is_available() else "cpu")
    device = torch.device('cpu')

    print('You will use : ',device)
    # Clustering result by scanpy
    # initial_pre = initial_clustering(RNA_matrix,custom_resolution=0.5,custom_n_neighbors=10)
    initial_pre = initial_clustering(RNA_matrix.copy())
    cluster_ini_num = len(set(initial_pre))
    ini_p1 = [int(i) for i in initial_pre]

    # Call the batch_select_whole function
    indices, Node_Ids, dic = batch_select_whole(RNA_matrix, labels, neighbor=[neighbor], cell_size=cell_size)
    # indices, Node_Ids, dic = batch_select_whole(RNA_matrix, label)
    sample_type = list(np.array(labels)[Node_Ids])

Cell number: 832
Gene number: 36601
You will use :  cpu
	When the number of cells is less than or equal to 500, it is recommended to set the resolution value to 0.2.
	When the number of cells is within the range of 500 to 5000, the resolution value should be set to 0.5.
	When the number of cells is greater than 5000, the resolution value should be set to 0.8.
         Falling back to preprocessing with `sc.pp.pca` and default params.
Partitioning data into batches based on sample type.


Processing Lymph Node samples: 100%|██████████| 21/21 [00:00<00:00, 27.71it/s]


### Model Training

- Obtain cell clustering results and attention-related matrices.

In [4]:
n_batch = len(indices)

# Reduce the dimensionality of features for cell, gene, and peak data.
node_model = NodeDimensionReduction(RNA_matrix, indices, ini_p1, n_hid=n_hid, n_heads=nheads, 
                                n_layers=nlayers, labsm=labsm, lr=lr, wd=wd, device=device, 
                                num_types=2, num_relations=1, epochs=10)
gnn = node_model.train_model(n_batch=n_batch)

# Tarin EmitGCL Model
EmitGCL_model = EmitGCL(gnn=gnn, labsm=labsm, n_hid=n_hid, n_batch=n_batch, device=device, lr=lr, wd=wd, 
                  pathway_genes=pathway_genes, gene_names=gene_names, sample_type=sample_type, num_epochs=20)
EmitGCL_gnn = EmitGCL_model.train_model(indices=indices, RNA_matrix=RNA_matrix, ini_p1=ini_p1, sample_type=sample_type, nodes_id=Node_Ids)

EmitGCL_result = EmitGCL_pred(RNA_matrix, EmitGCL_gnn=EmitGCL_gnn, indices=indices, nheads=nheads,
                            nodes_id=Node_Ids, cell_size=cell_size, device=device, 
                            gene_names=gene_names, node_dim_reduction_model=node_model)

The training process for the NodeDimensionReduction model has started. Please wait.


 10%|█         | 1/10 [01:14<11:14, 74.99s/it]

Epoch 1:
  KL Loss: 0.28742086835976305
  Cluster Loss: 3.684804603971284


 20%|██        | 2/10 [02:48<11:27, 85.95s/it]

Epoch 2:
  KL Loss: 0.2490111419866825
  Cluster Loss: 3.5531575433139144


...

100%|██████████| 29/29 [00:50<00:00,  1.73s/it]


Epoch 19:
  KL Loss: 0.7860991954803467
  Cluster Loss: 2.8021435737609863
  UCell Loss: 0.0
  Total Contrastive Loss: 0.0
  Total Loss: 3.588242769241333


100%|██████████| 29/29 [00:41<00:00,  1.44s/it]


Epoch 20:
  KL Loss: 0.7762186527252197
  Cluster Loss: 2.7814955711364746
  UCell Loss: -3.61641263961792
  Total Contrastive Loss: 0.0
  Total Loss: -0.058698415756225586
The training for the EmitGCL model has been completed.


100%|██████████| 29/29 [00:57<00:00,  1.98s/it]


### Save Model Output Results

- `EmitGCL_result['pred_label']`: Predicted cell clustering labels
- `EmitGCL_result['cell_embedding']`: Low-dimensional cell embeddings
- `EmitGCL_result['attention_weights']`: Cell-gene attention matrix

In [6]:
# Save Result
attention_matrices = EmitGCL_result['attention_weights']

for head, matrix in attention_matrices.items():
    save_attention_matrix_to_mtx(matrix, attention_file, head)

np.save(output_file + "pred.npy", EmitGCL_result['pred_label'])
np.save(output_file + "cell_embedding.npy", EmitGCL_result['cell_embedding'])